In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 500     # number of variables
num_ineq = 500    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 34831.17
Epoch 1, Validation Loss: 43.21
Epoch 2, Validation Loss: 35.74
Epoch 3, Validation Loss: 18.54
Epoch 4, Validation Loss: 26.63
Epoch 5, Validation Loss: 14.08
Epoch 6, Validation Loss: 8.38
Epoch 7, Validation Loss: 7.36
Epoch 8, Validation Loss: 10.26
Epoch 9, Validation Loss: 7.94
Epoch 10, Validation Loss: 16.69
Epoch 11, Validation Loss: 8.49
Epoch 12, Validation Loss: 7.56
Epoch 13, Validation Loss: 4.21
Epoch 14, Validation Loss: 2.82
Epoch 15, Validation Loss: 13.52
Epoch 16, Validation Loss: 4.26
Epoch 17, Validation Loss: 13.88
Epoch 18, Validation Loss: 5.60
Epoch 19, Validation Loss: 2.94
Epoch 20, Validation Loss: 1.20
Epoch 21, Validation Loss: 16.58
Epoch 22, Validation Loss: 1.21
Epoch 23, Validation Loss: 1.07
Epoch 24, Validation Loss: 0.96
Epoch 25, Validation Loss: 7.33
Epoch 26, Validation Loss: 3.42
Epoch 27, Validation Loss: 4.62
Epoch 28, Validation Loss: 0.90
Epoch 29, Validation Loss: 1.08
Epoch 30, Validation Loss: 6.26
Epoc

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:58<00:00,  1.70it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -0.716107          0.006552      0.006133
std      0.111679          0.039169      0.000881
min     -0.991856          0.000000      0.003006
25%     -0.759862          0.000000      0.005999
50%     -0.716757          0.000000      0.006210
75%     -0.653464          0.000000      0.006543
max     -0.380822          0.314706      0.008294
Number of infeasible solution: 4


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12499.07
Epoch 1, Validation Loss: 38.66
Epoch 2, Validation Loss: 23.40
Epoch 3, Validation Loss: 16.83
Epoch 4, Validation Loss: 34.34
Epoch 5, Validation Loss: 13.67
Epoch 6, Validation Loss: 12.04
Epoch 7, Validation Loss: 8.96
Epoch 8, Validation Loss: 16.09
Epoch 9, Validation Loss: 9.16
Epoch 10, Validation Loss: 4.83
Epoch 11, Validation Loss: 4.50
Epoch 12, Validation Loss: 10.73
Epoch 13, Validation Loss: 6.62
Epoch 14, Validation Loss: 3.60
Epoch 15, Validation Loss: 3.18
Epoch 16, Validation Loss: 6.19
Epoch 17, Validation Loss: 1.47
Epoch 18, Validation Loss: 1.09
Epoch 19, Validation Loss: 5.20
Epoch 20, Validation Loss: 1.87
Epoch 21, Validation Loss: 3.73
Epoch 22, Validation Loss: 1.36
Epoch 23, Validation Loss: 0.44
Epoch 24, Validation Loss: 1.26
Epoch 25, Validation Loss: 12.76
Epoch 26, Validation Loss: 1.45
Epoch 27, Validation Loss: 1.31
Epoch 28, Validation Loss: 0.30
Epoch 29, Validation Loss: 0.26
Epoch 30, Validation Loss: 0.81
Epoch

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:16<00:00,  1.31it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    -1.646320               0.0      0.006785
std      0.031313               0.0      0.001193
min     -1.675363               0.0      0.003505
25%     -1.656603               0.0      0.006344
50%     -1.656603               0.0      0.006558
75%     -1.655448               0.0      0.007088
max     -1.455971               0.0      0.010228
Number of infeasible solution: 0


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12553.04
Epoch 1, Validation Loss: -1.33
Epoch 2, Validation Loss: -16.12
Epoch 3, Validation Loss: -29.17
Epoch 4, Validation Loss: -37.21
Epoch 5, Validation Loss: -43.48
Epoch 6, Validation Loss: -47.20
Epoch 7, Validation Loss: -50.64
Epoch 8, Validation Loss: -53.48
Epoch 9, Validation Loss: -56.61
Epoch 10, Validation Loss: -61.43
Epoch 11, Validation Loss: -61.15
Epoch 12, Validation Loss: -63.67
Epoch 13, Validation Loss: -65.55
Epoch 14, Validation Loss: -65.49
Epoch 15, Validation Loss: -67.17
Epoch 16, Validation Loss: -66.86
Epoch 17, Validation Loss: -69.99
Epoch 18, Validation Loss: -69.55
Epoch 19, Validation Loss: -70.48
Epoch 20, Validation Loss: -68.71
Epoch 21, Validation Loss: -70.86
Epoch 22, Validation Loss: -72.14
Epoch 23, Validation Loss: -73.24
Epoch 24, Validation Loss: -70.37
Epoch 25, Validation Loss: -74.17
Epoch 26, Validation Loss: -74.73
Epoch 27, Validation Loss: -75.11
Epoch 28, Validation Loss: -77.63
Epoch 29, Validation Lo

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_pr_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:06<00:00,  1.50it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   9.302976e+01          4.344916      0.000933
std    4.284734e-14          1.152884      0.000526
min    9.302976e+01          1.866629      0.000000
25%    9.302976e+01          3.649780      0.000998
50%    9.302976e+01          4.132823      0.001001
75%    9.302976e+01          5.084770      0.001015
max    9.302976e+01          8.286500      0.002511
Number of infeasible solution: 100


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12499.07
Epoch 1, Validation Loss: 18.64
Epoch 2, Validation Loss: -1.59
Epoch 3, Validation Loss: -18.05
Epoch 4, Validation Loss: -21.73
Epoch 5, Validation Loss: -26.56
Epoch 6, Validation Loss: -27.22
Epoch 7, Validation Loss: -27.58
Epoch 8, Validation Loss: -34.26
Epoch 9, Validation Loss: -35.87
Epoch 10, Validation Loss: -38.61
Epoch 11, Validation Loss: -40.10
Epoch 12, Validation Loss: -41.03
Epoch 13, Validation Loss: -39.50
Epoch 14, Validation Loss: -38.63
Epoch 15, Validation Loss: -43.66
Epoch 16, Validation Loss: -45.00
Epoch 17, Validation Loss: -40.15
Epoch 18, Validation Loss: -44.84
Epoch 19, Validation Loss: -47.01
Epoch 20, Validation Loss: -45.50
Epoch 21, Validation Loss: -46.09
Epoch 22, Validation Loss: -45.65
Epoch 23, Validation Loss: -43.79
Epoch 24, Validation Loss: -46.50
Epoch 25, Validation Loss: -48.12
Epoch 26, Validation Loss: -50.34
Epoch 27, Validation Loss: -46.89
Epoch 28, Validation Loss: -46.52
Epoch 29, Validation Los

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_st_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:13<00:00,  1.37it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   1.048862e+02               0.0      0.003971
std    1.428245e-14               0.0      0.000962
min    1.048862e+02               0.0      0.001484
25%    1.048862e+02               0.0      0.003506
50%    1.048862e+02               0.0      0.004013
75%    1.048862e+02               0.0      0.004438
max    1.048862e+02               0.0      0.007432
Number of infeasible solution: 0
